# Import Stackup
This example shows how to import stackup file.

## Import the required packages

In [1]:
import json
from pathlib import Path
import tempfile

from IPython.display import display
import pandas as pd
from pyaedt.downloads import download_file

from pyedb import Edb

AEDT_VERSION = "2024.2"
NG_MODE = False


Download the example PCB data.

In [2]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", destination=temp_folder.name)

## Load example layout.

In [3]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

PyAEDT INFO: Logger is initialized in EDB.


PyAEDT INFO: legacy v0.25.0


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyAEDT INFO: Database ANSYS-HSD_V1.aedb Opened in 2024.2


PyAEDT INFO: Cell main Opened


PyAEDT INFO: Builder was initialized.


PyAEDT INFO: EDB initialized.


## Review original stackup definition

Get original stackup definition in a dictionary. Alternatively, stackup definition can be exported in a json file by
edbapp.configuration.export()

In [4]:
data_cfg = edbapp.configuration.get_data_from_db(stackup=True)

PyAEDT INFO: Getting data from layout database.


In [5]:
df = pd.DataFrame(data=data_cfg["stackup"]["layers"])
display(df)

,name,type,material,fill_material,thickness
0,1_Top,signal,copper,Solder Resist,0.000035
1,DE1,dielectric,Megtron4,,0.000100
2,Inner1(GND1),signal,copper,Megtron4_2,0.000017
3,DE2,dielectric,Megtron4_2,,0.000088
4,Inner2(PWR1),signal,copper,Megtron4_2,0.000017
5,DE3,dielectric,Megtron4,,0.000100
6,Inner3(Sig1),signal,copper,Megtron4_3,0.000017
7,Megtron4-1mm,dielectric,Megtron4_3,,0.001000
8,Inner4(Sig2),signal,copper,Megtron4_3,0.000017
9,DE5,dielectric,Megtron4,,0.000100


## Modify stackup

Modify top layer thickness

In [6]:
data_cfg["stackup"]["layers"][0]["thickness"] = 0.00005

Add a solder mask layer

In [7]:
data_cfg["stackup"]["layers"].insert(
    0, {"name": "soler_mask", "type": "dielectric", "material": "Megtron4", "fill_material": "", "thickness": 0.00002}
)

Review modified stackup

In [8]:
df = pd.DataFrame(data=data_cfg["stackup"]["layers"])
display(df.head(3))

,name,type,material,fill_material,thickness
0,soler_mask,dielectric,Megtron4,,0.00002
1,1_Top,signal,copper,Solder Resist,0.00005
2,DE1,dielectric,Megtron4,,0.00010


Write stackup definition into a json file

In [9]:
file_cfg = Path(temp_folder.name) / "edb_configuration.json"
with open(file_cfg, "w") as f:
    json.dump(data_cfg, f, indent=4, ensure_ascii=False)

## Load stackup from json configuration file

In [10]:
edbapp.configuration.load(file_cfg, apply_file=True)

Plot stackup

In [11]:
edbapp.stackup.plot()

C:\actions-runner\_work\pyedb\pyedb\.venv\lib\site-packages\pyedb\dotnet\edb_core\stackup.py:2862: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


<module 'matplotlib.pyplot' from 'C:\\actions-runner\\_work\\pyedb\\pyedb\\.venv\\lib\\site-packages\\matplotlib\\pyplot.py'>

Check top layer thickness

In [12]:
edbapp.stackup["1_Top"].thickness

5e-05

## Cleanup

In [13]:
edbapp.close()

True